## Calculating Transfer Function between two TE accelerometers with a labJack T7-Pro
This is an attempt to calculate the transfer function from the stored\
accelerations of two accelerometers.  It estimates the TF as the ratio of the\
cross spectral densities H(k) = Pyy(k) / Pxy(k)*, as in this link:\
https://www.mathworks.com/help/signal/ref/tfestimate.html#bufqg8e

Craig Lage - Sep 27, 2021


In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import lsim, csd
from scipy.optimize import minimize

In [ ]:
# Read in the pickled accelerometer data
scanRate = 1000
readTime = 10.0
df = pd.read_pickle("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.pkl")

In [ ]:
# Plot the data at different time magnifications.
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

for n, scanIndex in enumerate([[0, int(scanRate*readTime-1)], [450, 750], [500,600]]):

    sub_df = df[df.index[scanIndex[0]] : df.index[scanIndex[1]]]
    
    plt.subplot(2,3,n+1)
    ax1 = sub_df['a1z'].plot(label="a1z", color='red')
    ax1.set_title("Axis 1", fontsize=16)
    ax1.set_ylabel("Acceleration(g)")
    ax1.legend(loc='center left')

    plt.subplot(2,3,n+4)
    ax2 = sub_df['a2z'].plot(label="a2z", color='red')
    ax2.set_title("Axis 2", fontsize=16)
    ax2.set_ylabel("Acceleration(g)")
    ax2.legend(loc='center left')
    
    if n == 0: # save this data for the FFT
        a1z = np.array(sub_df['a1z'].tolist())
        a2z = np.array(sub_df['a2z'].tolist())

#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_14Sep21.pdf")

In [ ]:
# Estimate the TF as  H(k) = Pyy(k) / Pxy(k)*  as discussed above
# Plot it several ways

Pyy = csd(a2z, a2z, fs=1000)
Pxy = csd(a1z, a2z, fs=1000)
tf =  Pyy[1] / np.conj(Pxy[1]) # H2 Pyy / Pxy*
plt.figure(figsize=(8,4))
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.title("Magnitude(TF)", fontsize=24)
plt.plot(Pyy[0], np.abs(tf), marker='x', color='blue', label = 'TF')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("|TF|", fontsize=16)
plt.xlim(20,200)
plt.ylim(0,4.0)
plt.subplot(1,2,2)
plt.title("Phase(TF)", fontsize=24)
plt.plot(Pyy[0], np.angle(tf, deg=True), marker='x', color='blue', label = 'TF')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("$ \phi $ (degrees)", fontsize=16)
plt.xlim(20,200)

## Estimating a functional form for the transfer function.
The next step is to try and find a functional form for the transfer function in a form like this:\
$ H(s) = \frac{b1 s + b0}{s^2 + a1 s + a0} $\
Once this is done, scipy routines like scipy.signal.lsim and scipy.signal.TransferFunction can be used.\
The function calcTransferFunction finds the coefficients that maximize the fit.

In [ ]:
def calcTransferFunction(freq, tf, fMin=10.0, fMax=200.0, param0=[1.0,1.0,1.0,1.0]):
    # Calculates a transfer function in polynomial form (2nd order only)
    # given an estimate as a function of frequency
    # freq is the list of frequencies
    # tf is the transfer function estimate
    # fMin and fMax are the min an dmax frequencies for the optimization
    # and param0 is an initial bext guess.
    
    def Deviation(params):
        # Finds the model vs estimate deviation
        result = 0.0
        [b1, b0, a1, a0] = params
        N = len(freq)

        def H(k):
            s = 2.0 * np.pi * 1j * k
            H = (b1 * s + b0) / (s * s + a1 * s + a0)
            return H

        for i in range(N):
            k = freq[i]
            if (k < fMin) or (k > fMax):
                continue
            diff = H(k) - tf[i]
            result += np.abs(diff)
        return result
    Result = minimize(Deviation, param0, method='Powell')
    #print(Result)
    #print(Result.x)
    num = [Result.x[0], Result.x[1]]
    den = [1.0, Result.x[2], Result.x[3]]
    #print(num)
    #print(den)
    return [num, den]


In [ ]:
# Now plot the best fit transfer function.
# To estimate the starting coefficients, we note that there is a pole at ~ 100Hz
# and that the output signal is about 0.5 * the input
a0 = (2.0 * np.pi * 100.0)**2.0
[num, den] = calcTransferFunction(Pyy[0], tf, param0=[1.0, a0/2.0, 10.0, a0])
s = 2.0 * np.pi * 1j * Pyy[0]
H = (num[0] * s + num[1]) / (den[0] * s * s + den[1] * s + den[2])

plt.figure(figsize=(12,4))
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.title("Magnitude(TF)", fontsize=24)
plt.plot(Pyy[0], np.abs(tf), marker='x', color='blue', label = 'TF estimate')
plt.plot(Pyy[0], np.abs(H), color = 'red', lw = 2.0, label = f'H(s) = \n ({num[0]:.1f}*s + {num[1]:.3g}) /\n ({den[0]:.1f}*s^2 + {den[1]:.1f}*s + {den[2]:.3g})')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("|TF|", fontsize=16)
plt.xlim(20,200)
plt.ylim(0,4.0)
plt.legend()
plt.subplot(1,2,2)
plt.title("Phase(TF)", fontsize=24)
plt.plot(Pyy[0], np.angle(tf, deg=True), marker='x', color='blue', label = 'TF estimate')
plt.plot(Pyy[0], np.angle(H, deg=True), color = 'red', lw = 2.0, label = f'H(s) = \n ({num[0]:.1f}*s + {num[1]:.3g}) /\n ({den[0]:.1f}*s^2 + {den[1]:.1f}*s + {den[2]:.3g})')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("$ \phi $ (degrees)", fontsize=16)
plt.xlim(20,200)
plt.legend()
#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Function_Fit_29Sep21.pdf")

In [ ]:
# Now use the calculated transfer function to predict the output from the input, using scipy.signal.lsim
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

for n, scanIndex in enumerate([[510, 710], [2180, 2380], [5040, 5240]]):

    sub_df = df[df.index[scanIndex[0]] : df.index[scanIndex[1]]]
    a1z = np.array(sub_df['a1z'].tolist())
    a1z = a1z - a1z.mean()
    a2z = np.array(sub_df['a2z'].tolist())
    a2z = a2z - a2z.mean()
    times = np.arange(0, (len(sub_df))/scanRate, 1.0/scanRate)
    plt.subplot(2,3,n+1)
    plt.plot(times, a1z)
    plt.title("Input", fontsize=16)
    plt.ylabel("Acceleration(g)")
    plt.xlabel("Time(sec)")

    outSim = lsim((num, den), a1z, times)

    plt.subplot(2,3,n+4)
    plt.plot(times, a2z, label = "Measured")
    plt.plot(times, outSim[1], label = "Simulated")
    plt.title("Output", fontsize=16)
    plt.ylabel("Acceleration(g)")
    plt.xlabel("Time(sec)")
    plt.legend(loc='lower center')
#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/TF_Simulated_Output_29Sep21.pdf")

In [ ]:
# Looking at the poles and zeros

pole_root = (den[1] + np.sqrt(-den[1]**2 + 4.0 * den[2]) * 1j) / 2.0
#print(pole_root)
# This is the frequency of the pole
pole_freq = np.imag(pole_root) / (2.0 * np.pi)
print(f"Pole frequency = {pole_freq:.3f} Hz")

zero_root = num[1] / num[0]
#print(zero_root)
# This is the frequency of the zero
zero_freq = zero_root / (2.0 * np.pi)
print(f"Zero frequency = {zero_freq:.3f} Hz")